In [1]:
import pandas as pd
import os
import shutil
import numpy as np

In [2]:
new_df = pd.read_csv('df_part_4.csv')
new_dataset_path = 'images_backup_4'
original_dataset_dir = '/tf/pstage_1/train/images_merge_3/'
original_csv_dir = '/tf/pstage_1/train/train.csv'

In [3]:
new_df

,ID,TYPE,USER_ID,GENDER,AGE,name,size_mb
0,30001,1,1092,MALE,19,030001_1_001092_MALE_19.jpg,0.933037
1,30001,2,1092,MALE,19,030001_2_001092_MALE_19.jpg,0.975251
2,30001,3,1092,MALE,19,030001_3_001092_MALE_19.jpg,1.027843
3,30001,4,1092,MALE,19,030001_4_001092_MALE_19.jpg,1.040692
4,30002,1,5794,MALE,30,030002_1_005794_MALE_30.jpg,1.503981
...,...,...,...,...,...,...,...
39995,39999,4,20782,FEMALE,22,039999_4_020782_FEMALE_22.jpg,5.155726
39996,40000,1,20783,MALE,33,040000_1_020783_MALE_33.jpg,1.030267
39997,40000,2,20783,MALE,33,040000_2_020783_MALE_33.jpg,1.061272
39998,40000,3,20783,MALE,33,040000_3_020783_MALE_33.jpg,0.987479


In [4]:
drop_id = set(new_df[new_df.size_mb < 0.1].ID.values)
len(drop_id)

26

In [5]:
drop_index_list = []
for id_ in drop_id:
    drop_index = new_df[new_df.ID == id_].index
    for d_idx in drop_index:
        drop_index_list.append(d_idx)

len(drop_index_list)

104

In [6]:
new_df = new_df.drop(drop_index_list)
len(new_df)

39896

- 100kB 미만 크기의 사진 제외 (너무 작은 사진이 있을 수 있음)

In [7]:
new_df = new_df[new_df.AGE < 100]
len(new_df)

39824

- 100살 이상 데이터 제거 (117, 2019, 2020 존재)

In [8]:
new_df.GENDER.value_counts()

FEMALE    16048
MALE      15244
NONE       8532
Name: GENDER, dtype: int64

In [9]:
new_df = new_df[new_df.GENDER != 'NONE']
len(new_df)

31292

- NONE 성별 데이터 제거

In [10]:
new_df.GENDER.str.lower().values

array(['male', 'male', 'male', ..., 'male', 'male', 'male'], dtype=object)

In [11]:
id_val = new_df.ID.values
gender = new_df.GENDER.str.lower().values
age = new_df.AGE.values
img_name = new_df.name.values
mask_type = new_df.TYPE.values


In [12]:
len_data = len(id_val)
print(len_data) #사용할 데이터

31292


In [13]:
## New data frame series
series_id = []
series_age = []
series_gender = []
dir_name_list = []

In [14]:
for idx in range(len_data):
    
    # Person directory generation
    if idx % 4 == 0:
        dir_name = str(id_val[idx]) + '_' + gender[idx] + '_' + 'Unknown' + '_' + str(age[idx])
        img_dir_path = os.path.join(new_dataset_path, dir_name)
        os.makedirs(img_dir_path, exist_ok = True)
        
        ## No same name
        series_id.append(id_val[idx])
        series_gender.append(gender[idx])
        series_age.append(age[idx])
        dir_name_list.append(dir_name)
        ## No same name
    

    # Image name change
    if mask_type[idx] == 1:
        img_name_change = 'mask.jpg'
    elif mask_type[idx] == 2:
        img_name_change = 'incorrect_mask1.jpg'
    elif mask_type[idx] == 3:
        img_name_change = 'incorrect_mask2.jpg'
    elif mask_type[idx] == 4:
        img_name_change = 'normal.jpg'
    
    # move image file to person directory
    original_img_path = os.path.join(new_dataset_path, img_name[idx])
    new_img_path = os.path.join(img_dir_path, img_name_change)
    shutil.move(original_img_path, new_img_path)


In [15]:
len(dir_name_list)

7823

In [16]:
new_dataset_df = pd.DataFrame({
                        'id': series_id,
                        'gender': series_gender,
                        'race': ['Unknown'] * int(len_data/4),
                        'age': series_age,
                        'path': dir_name_list
})

In [17]:
new_dataset_df.id = new_dataset_df.id.astype(object)

In [18]:
new_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7823 entries, 0 to 7822
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7823 non-null   object
 1   gender  7823 non-null   object
 2   race    7823 non-null   object
 3   age     7823 non-null   int64 
 4   path    7823 non-null   object
dtypes: int64(1), object(4)
memory usage: 305.7+ KB


In [19]:
original_df = pd.read_csv(original_csv_dir)

In [20]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700 entries, 0 to 2699
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2700 non-null   object
 1   gender  2700 non-null   object
 2   race    2700 non-null   object
 3   age     2700 non-null   int64 
 4   path    2700 non-null   object
dtypes: int64(1), object(4)
memory usage: 105.6+ KB


In [21]:
merged_df = pd.concat([new_dataset_df, original_df])
merged_df

,id,gender,race,age,path
0,30001,male,Unknown,19,30001_male_Unknown_19
1,30002,male,Unknown,30,30002_male_Unknown_30
2,30003,female,Unknown,36,30003_female_Unknown_36
3,30004,male,Unknown,28,30004_male_Unknown_28
4,30005,male,Unknown,21,30005_male_Unknown_21
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [22]:
#### 위의 코드를 다 돌리고 실행해야 합니다!!
#### 위의 코드를 다 돌리고 실행해야 합니다!!

#디렉토리명 직접 수정해주세요..
!rm images_backup_4/*.jpg

#### 위의 코드를 다 돌리고 실행해야 합니다!!
#### 위의 코드를 다 돌리고 실행해야 합니다!!

- AGE 100이상, GENDER NONE 이미지들을 제거합니다

In [23]:
merged_df.to_csv('merged_df.csv', index = False)

In [24]:
for person in os.listdir(new_dataset_path):
    person_dir = os.path.join(new_dataset_path, person)

    
    shutil.move(person_dir, original_dataset_dir)


- 이미지가 담긴 디렉토리들을 기존 데이터가 있는 디렉토리로 옮깁니다.

### 이후 생성된 merged_df.csv 파일을 기존 훈련 코드에서 읽어와서 데이터셋을 만들면 되겠습니다.